In [1]:
# %pip install pydantic openai tqdm pandas -U

In [1]:
# %pip install jinja2 openpyxl

In [17]:
import json
from typing import Literal
from openai import OpenAI
from pydantic import BaseModel, Field
import pandas as pd
import openai
import time

from tqdm import tqdm

In [ ]:
api_key = '<api-key>'

In [5]:
client = OpenAI(api_key=api_key)

In [6]:
model_str="gpt-4o-2024-11-20"

## Accuracy scoring

In [77]:
system_prompt = """
Ты помощник врача офтальмолога. Извлеки параметры глазного дна по его описанию. Пиши значения всех параметров в начальной форме.
"""


In [78]:
from typing import Optional
from pydantic import BaseModel, Field

class EyeFundusEvaluation(BaseModel):
    od_color: str = Field(..., description="Цвет диска зрительного нерва, например, серый, бледный или белый.")
    od_monotone: str = Field(..., description="Имеет ли диск зрительного нерва монотонный (однородный) цвет. Пиши наблюдается или не наблюдается.")
    od_size: str = Field(..., description="Размер диска зрительного нерва: нормальный, больше нормы или меньше нормы.")
    od_shape: str = Field(..., description="Форма диска зрительного нерва: правильная, овальная или неправильная.")
    od_border: str = Field(..., description="Чёткость границ диска зрительного нерва: четкие или размытые.")
    od_excavation_size: str = Field(..., description="Размер экскавации на диске зрительного нерва: нормальный, больше нормы или меньше нормы.")
    od_excavation_location: str = Field(..., description="Расположение экскавации: в центре, верхний, нижний, наружный, внутренний и др.")
    od_excavation_ratio: str = Field(..., description="Отношение размеров экскавации к диску Э/Д.")
    od_vessels_location: str = Field(..., description="Расположение сосудистого пучка на диске зрительного нерва.")

    vessels_art_course: str = Field(..., description="Направление или смещение артерий: нормальное, наружное или внутреннее смещение.")
    vessels_art_turtuosity: str = Field(..., description="Извитость артерий: нормальная, извитая или прямая.")
    vessels_art_bifurcation: str = Field(..., description="Угол бифуркации артерий: нормальная, под острым углом или под тупым углом.")
    vessels_art_caliber: str = Field(..., description="Калибр артерий: нормальный, расширенный или суженный.")

    vessels_vein_course: str = Field(..., description="Направление или смещение вен: нормальное, наружное или внутреннее смещение.")
    vessels_vein_turtuosity: str = Field(..., description="Извитость вен: нормальная, извитая или прямая.")
    vessels_vein_bifurcation: str = Field(..., description="Угол бифуркации вен: нормальная, под острым углом или под тупым углом.")
    vessels_vein_caliber: str = Field(..., description="Калибр вен: нормальный, расширенный или суженный.")

    vessels_ratio: str = Field(..., description="Соотношение артерий и вен, показатель А/В.")

    macula_macular_reflex: str = Field(..., description="Состояние макулярного рефлекса: нормальный, гладкий или отсутствует.")
    macula_foveal_reflex: str = Field(..., description="Состояние фовеального рефлекса: нормальный, гладкий или отсутствует.")


### **Scoring**

In [80]:
def scoring_function(data_path, type):
    df = pd.read_excel(data_path)
    texts = df['text']
    dataframe_elements = []

    for text in tqdm(texts, type):
        success = False
        while not success:
            try:
                completion = client.beta.chat.completions.parse(
                    model=model_str,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": text},
                    ],
                    response_format=EyeFundusEvaluation,
                )
                success = True
                json_answer = completion.choices[0].message.content
                #print(json_answer)
                llm_dict = json.loads(json_answer)
                dataframe_elements.append(llm_dict)
            except openai.RateLimitError as e:
                # print(f"Rate limit exceeded. Waiting 1 second. Details: {e}")
                time.sleep(30)
        

    df = pd.DataFrame(dataframe_elements)
    df.to_csv(f"scores_v2/{type}_scores.csv")

In [ ]:
scoring_metas = [
    ('data_v2/Yandex_data_v2.xlsx', 'YandexGPT'),
    ('data_v2/Saiga_Llama3_data_v2.xlsx', 'Saiga_Llama'),
    ('data_v2/T_lite_data_v2.xlsx', 'T-lite'),
    ('data_v2/Gemma3_4b_data_v2.xlsx', 'Gemma3_4b'),
    ('data_v2/RuAdapt_Qwen_data_v2.xlsx', 'RuAdapt_Qwen'),
    ('data_v2/Saiga_Gemma3_12b_data_v2.xlsx', 'Saiga_Gemma3_12b'), 
    ('data_v2/TableLlama_data_v2.xlsx', 'TableLlama'),
]

for meta in scoring_metas:
    print(meta)
    scoring_function(meta[0], meta[1])

('data_v2/Saiga_Gemma3_12b_data_v2.xlsx', 'Saiga_Gemma3_12b')


Saiga_Gemma3_12b: 100%|██████████| 125/125 [17:52<00:00,  8.58s/it]


('data_v2/TableLlama_data_v2.xlsx', 'TableLlama')


TableLlama: 100%|██████████| 125/125 [18:54<00:00,  9.08s/it]


## Report Order, Language, Structure

In [7]:
system_prompt = """
Ты выступаешь в роли судьи для оценки качества генерации текста заключения офтальмолога, созданного языковой моделью на основании параметров глазного дна. Проверь ответ по четырём критериям, выстави оценки и обоснуй их.

**Критерий №1: Отсутствие вводных конструкций и лишних фраз**  
Текст должен содержать только медицинское описание. Не допускаются вводные конструкции или пояснительные фразы (например: "конечно", "текст описания", "ответ на ваш вопрос" и т.п.).

**Оценка:**  
- 1 балл — нет вводных конструкций  
- 0 баллов — есть хотя бы одна

Приведи пример лишней фразы при оценке 0.

---

**Критерий №2: Соблюдение порядка описания структур**  
Порядок должен быть следующим:

1. **Диск зрительного нерва (ДЗН)**  
   - Цвет  
   - Монотонность окраски  
   - Размер  
   - Форма  
   - Границы  

2. **Экскавация ДЗН**  
   - Размер экскавации  
   - Положение экскавации 
   - Соотношение Э/Д 

3. **Сосудистый пучок**

4. **Сосуды глазного дна (Артерии и Вены)**  
   - Ход  
   - Извитость  
   - Бифуркация  
   - Калибр  
   - А/В индекс  

5. **Макула**  
   - Макулярный рефлекс  
   - Фовеолярный рефлекс

**Оценка:**  
- 1 балл — порядок строго соблюдён  
- 0 баллов — есть отклонения

Укажи, где нарушен порядок при оценке 0.

**Пример 1**
ДЗН бледно-розовый, монотонность не наблюдается, диск нормального размера и правильной формы, границы диска четкие. Экскавация диска нормального размера, расположена в центре, соотношение э/д = 0.3. Сосудистый пучок расположен в центре.  
Сосуды имеют нормальный ход и нормальную извитость, бифуркация и калибр в норме, соотношение калибра артерий и вен 2/3. 
Макулярный и фовеолярные рефлексы четкие.

Оценка: 1 балл - порядок описания строго соблюдён.

**Пример 2**
ДЗН розовый, границы чёткие, форма овальная, размер меньше нормы. Экскавация нормальная, в верхне-наружном секторе. Сосудистый пучок расположен внутренне.  
Артерии: ход нормальный, извитость прямая, бифуркация под тупым углом, калибр суженный.  
Вены: ход смещён наружу, извитость прямая, бифуркация под острым углом, калибр расширенный.  
А/В индекс 2:3.  
Макулярный рефлекс сглаженный, фовеальный рефлекс нормальный.

Оценка: 0 баллов - порядок описания нарушен, описание размера ДЗН идет после описания границ и формы. Описание монотонности опущено.

---

**Критерий №3: Плавность и естественность языка**  
Оцени текст с точки зрения профессионального языка врача-офтальмолога. Текст должен быть грамотным, хорошо читаемым.

**Оценка:**  
От 0 (очень неестественно) до 5 (максимально естественно и профессионально)

Кратко обоснуй оценку, указав стилистически слабые места, если есть.
"""


In [8]:
class EvaluationResult(BaseModel):
    no_intro_phrases_score: Literal[0, 1] = Field(..., description="1, если в тексте нет вводных конструкций и лишних фраз; 0 — если есть хотя бы одна")
    structure_order_score: Literal[0, 1] = Field(..., description="1, если порядок описания структур соблюдён строго; 0 — если есть отклонения")
    language_naturalness_score: Literal[0, 1, 2, 3, 4, 5] = Field(..., description="Оценка от 0 до 5 за плавность и естественность медицинского языка")

    no_intro_phrases_comment: str = Field(..., description="Комментарий к оценке по вводным конструкциям; привести пример при необходимости")
    structure_order_comment: str = Field(..., description="Комментарий к оценке порядка структур; указать, где нарушен порядок при необходимости")
    language_naturalness_comment: str = Field(..., description="Комментарий к оценке плавности и естественности языка; обосновать балл")

In [9]:
def scoring_function(data_path, type):
    df = pd.read_excel(data_path)
    texts = df['text']
    dataframe_elements = []

    for text in tqdm(texts, type):
        success = False
        while not success:
            try:
                completion = client.beta.chat.completions.parse(
                    model=model_str,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": text},
                    ],
                    response_format=EvaluationResult,
                )
                success = True
                json_answer = completion.choices[0].message.content
                #print(json_answer)
                llm_dict = json.loads(json_answer)
                dataframe_elements.append(llm_dict)
            except openai.RateLimitError as e:
                # print(f"Rate limit exceeded. Waiting 1 second. Details: {e}")
                time.sleep(30)
        

    df = pd.DataFrame(dataframe_elements)
    df.to_csv(f"scores_v3/{type}_scores.csv")

In [15]:
scoring_metas = [
    ('data_v2/Yandex_data_v2.xlsx', 'YandexGPT'),
    ('data_v2/T_lite_data_v2.xlsx', 'T-lite'),
    ('data_v2/Saiga_Llama3_data_v2.xlsx', 'Saiga_Llama'),
    ('data_v2/Gemma3_4b_data_v2.xlsx', 'Gemma3_4b'),
    ('data_v2/RuAdapt_Qwen_data_v2.xlsx', 'RuAdapt_Qwen'),
    ('data_v2/Saiga_Gemma3_12b_data_v2.xlsx', 'Saiga_Gemma3_12b'), 
    ('data_v2/TableLlama_data_v2.xlsx', 'TableLlama'),
]

for meta in scoring_metas:
    print(meta)
    scoring_function(meta[0], meta[1])

('data_v2/TableLlama_data_v2.xlsx', 'TableLlama')


TableLlama:   0%|          | 0/125 [00:00<?, ?it/s]

TableLlama: 100%|██████████| 125/125 [09:28<00:00,  4.54s/it]


In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
with open("TEST_samples_v2.json", "r", encoding="utf-8") as f:
    loaded_samples = json.load(f)

In [ ]:
df_init = pd.DataFrame(loaded_samples)
df_init = df_init.map(lambda x: x.lower().replace('ё', 'е') if isinstance(x, str) else x)

In [ ]:
df_init = df_init[['od_color', 'od_monotone', 'od_size', 'od_shape', 'od_border',
        'od_excavation_size', 'od_excavation_location', 'od_vessels_location',
        'od_excavation_ratio', 'vessels_art_course', 'vessels_art_turtuosity',
        'vessels_art_bifurcation', 'vessels_art_caliber', 'vessels_vein_course',
        'vessels_vein_turtuosity', 'vessels_vein_bifurcation',
        'vessels_vein_caliber', 'vessels_ratio', 'macula_macular_reflex',
        'macula_foveal_reflex']]

##  Accuracy scoring results

In [ ]:
scoring_metas = [
    ('scores_v2/YandexGPT_scores.csv', 'YandexGPT'),
    ('scores_v2/Saiga_Llama_scores.csv', 'Saiga_Llama'),
    ('scores_v2/T-lite_scores.csv', 'T-lite'),
    ('scores_v2/Gemma3_4b_scores.csv', 'Gemma3_4b'),
    ('scores_v2/RuAdapt_Qwen_scores.csv', 'RuAdapt_Qwen'),
    ('scores_v2/Saiga_Gemma3_12b_scores.csv', 'Saiga_Gemma3_12b'), 
    ('scores_v2/TableLlama_scores.csv', 'TableLlama'),
]

In [ ]:
for meta in scoring_metas:
    df = pd.read_csv(meta[0])
    df = df.drop(['Unnamed: 0'], axis=1)
    df = df.map(lambda x: x.lower().replace('ё', 'е') if isinstance(x, str) else x)
    try:
        df['od_excavation_ratio'] = df['od_excavation_ratio'].astype(str)
        srs = df['od_excavation_ratio'].apply(lambda x: x.replace(',', '.')).astype(float)
        df['od_excavation_ratio'] = srs
    except:
        df['od_excavation_ratio'] = df['od_excavation_ratio'].apply(lambda x: x.replace(',', '.'))

    df = df[['od_color', 'od_monotone', 'od_size', 'od_shape', 'od_border',
        'od_excavation_size', 'od_excavation_location', 'od_vessels_location',
        'od_excavation_ratio', 'vessels_art_course', 'vessels_art_turtuosity',
        'vessels_art_bifurcation', 'vessels_art_caliber', 'vessels_vein_course',
        'vessels_vein_turtuosity', 'vessels_vein_bifurcation',
        'vessels_vein_caliber', 'vessels_ratio', 'macula_macular_reflex',
        'macula_foveal_reflex']]

    mean_acc = np.round(np.mean((df == df_init).sum() / 125), 3)
    print(f"Mean accuracy for {meta[1]} : {mean_acc}")

Mean accuracy for YandexGPT : 0.735
Mean accuracy for Saiga_Llama : 0.634
Mean accuracy for T-lite : 0.694
Mean accuracy for Gemma3_4b : 0.198
Mean accuracy for RuAdapt_Qwen : 0.582
Mean accuracy for Saiga_Gemma3_12b : 0.75
Mean accuracy for TableLlama : 0.248


## Order, Language, Structure scoring results

In [ ]:
scoring_metas = [
    ('scores_OLS/YandexGPT_scores.csv', 'YandexGPT'),
    ('scores_OLS/Saiga_Llama_scores.csv', 'Saiga_Llama'),
    ('scores_OLS/T-lite_scores.csv', 'T-lite'),
    ('scores_OLS/Gemma3_4b_scores.csv', 'Gemma3_4b'),
    ('scores_OLS/RuAdapt_Qwen_scores.csv', 'RuAdapt_Qwen'),
    ('scores_OLS/Saiga_Gemma3_12b_scores.csv', 'Saiga_Gemma3_12b'), 
    ('scores_OLS/TableLlama_scores.csv', 'TableLlama'),
]

In [ ]:
for meta in scoring_metas:
    print(f"Model {meta[1]}")

    df = pd.read_csv(meta[0])
    df = df.drop(['Unnamed: 0'], axis=1)
    df_scores = df[['no_intro_phrases_score', 'structure_order_score', 'language_naturalness_score']]
    display(df_scores.mean()) 

Model YandexGPT


no_intro_phrases_score        1.000
structure_order_score         0.656
language_naturalness_score    4.432
dtype: float64

Model Saiga_Llama


no_intro_phrases_score        0.912
structure_order_score         0.432
language_naturalness_score    4.176
dtype: float64

Model T-lite


...

no_intro_phrases_score        0.976
structure_order_score         0.360
language_naturalness_score    4.224
dtype: float64

Model Saiga_Gemma3_12b


no_intro_phrases_score        1.000
structure_order_score         0.648
language_naturalness_score    4.496
dtype: float64

Model TableLlama


no_intro_phrases_score        0.304
structure_order_score         0.016
language_naturalness_score    2.168
dtype: float64

## Pathology

In [ ]:
df_yndx = pd.read_excel('pathology/Yandex_data_pathlgy_v2.xlsx')

In [ ]:
with open("pathology/TEST_samples_pathlgy.json", "r", encoding="utf-8") as f:
    loaded_samples = json.load(f)

In [ ]:
df_init = pd.DataFrame(loaded_samples)
df_init.head()
df_pathlgy = df_init[['od_pathology', 'vessels_pathology', 'macula_pathology', 'peripheral_pathology']]
df_pathlgy = df_pathlgy.join(df_yndx)
df_pathlgy.to_excel('pathology/Yandex_pathology_full.xlsx', index=False)

In [ ]:
df_pathlgy.head()

,od_pathology,vessels_pathology,macula_pathology,peripheral_pathology,text,time
0,К наружи от экскавации темно-серый очаг с четк...,По ходу сосудов единичные шарообразные утолщен...,Дополнительно присутствуют кольцевидные рефлек...,По всему глазному дну множественные очаги крас...,"ДЗН бледно-розовый, наблюдается монотонность, ...",11.228165
1,К наружи от экскавации темно-серый очаг с четк...,По ходу сосудов единичные шарообразные утолщен...,Дополнительно присутствуют кольцевидные рефлек...,"Единичный очаг отслойки сетчатки на периферии,...","ДЗН бледно-розовый, наблюдается монотонность, ...",6.386166
2,К наружи от экскавации темно-серый очаг с четк...,По ходу сосудов единичные шарообразные утолщен...,Дополнительно присутствуют кольцевидные рефлек...,Множественные мелкие пигментированные очаги кр...,"ДЗН бледно-розовый, наблюдается монотонность, ...",6.578877
3,К наружи от экскавации темно-серый очаг с четк...,По ходу сосудов единичные шарообразные утолщен...,В макуле видны крупные пузырьки (кисты) с проз...,По всему глазному дну множественные очаги крас...,"ДЗН бледно-розовый, наблюдается монотонность, ...",6.555842
4,К наружи от экскавации темно-серый очаг с четк...,По ходу сосудов единичные шарообразные утолщен...,В макуле видны крупные пузырьки (кисты) с проз...,"Единичный очаг отслойки сетчатки на периферии,...","ДЗН бледно-розовый, наблюдается монотонность, ...",6.725252
